In [1]:
import os

In [2]:
%pwd

'/Users/rahatkapoor/Desktop/ML Projects/Text summarizer/Text-summarizer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/rahatkapoor/Desktop/ML Projects/Text summarizer/Text-summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
import sys
from pathlib import Path

# Assuming your notebook is located in 'Text-summarizer/research'
current_dir = Path.cwd()  # Get the current working directory of the notebook
project_dir = current_dir.parent  # Go up two levels to reach 'Text-summarizer'

# Add 'src' directory to sys.path
src_dir = project_dir / 'src'
sys.path.insert(0, str(src_dir))

# Now you can import from your package
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
from pathlib import Path

class ConfigurationManager:
    def __init__(
        self,
        config_filepath='/Users/rahatkapoor/Desktop/ML Projects/Text summarizer/Text-summarizer/config/config.yaml',
        params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(Path(config_filepath))  # Convert to Path object
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.evaluation_strategy,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps
        )

        return model_trainer_config


In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

/Users/rahatkapoor/anaconda3/envs/textS/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-06-25 14:12:46,964: INFO: config: PyTorch version 2.3.1 available.]


In [9]:
import os
import torch
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from datasets import load_from_disk

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"

        # Load tokenizer and model
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)

        # Data collator for seq2seq tasks
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        # Load dataset
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # Training arguments
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        # Create Trainer instance
        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["train"], 
            eval_dataset=dataset_samsum_pt["validation"]
        )
        
        # Start training
        trainer.train()

        # Save model and tokenizer
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


In [10]:
Path.cwd()

PosixPath('/Users/rahatkapoor/Desktop/ML Projects/Text summarizer/Text-summarizer')

In [11]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-06-25 14:12:47,125: INFO: common: yaml file: /Users/rahatkapoor/Desktop/ML Projects/Text summarizer/Text-summarizer/config/config.yaml loaded successfully]
[2024-06-25 14:12:47,127: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-25 14:12:47,128: INFO: common: created directory at: artifacts]


BoxKeyError: "'ConfigBox' object has no attribute 'trainingarguments'"